# Scraping

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# Requête et parsing de la page souhaitée
# Nous chargeons ici une page un peu plus compliquée
page = requests.get("http://dataquestio.github.io/web-scraping-pages/ids_and_classes.html")
soup = BeautifulSoup(page.content)
soup

<html>
<head>
<title>A simple example page</title>
</head>
<body>
<div>
<p class="inner-text first-item" id="first">
                First paragraph.
            </p>
<p class="inner-text">
                Second paragraph.
            </p>
</div>
<p class="outer-text first-item" id="second">
<b>
                First outer paragraph.
            </b>
</p>
<p class="outer-text">
<b>
                Second outer paragraph.
            </b>
</p>
</body>
</html>

In [5]:
print(page)

<Response [200]>


In [3]:
print(soup.prettify())

<html>
 <head>
  <title>
   A simple example page
  </title>
 </head>
 <body>
  <div>
   <p class="inner-text first-item" id="first">
    First paragraph.
   </p>
   <p class="inner-text">
    Second paragraph.
   </p>
  </div>
  <p class="outer-text first-item" id="second">
   <b>
    First outer paragraph.
   </b>
  </p>
  <p class="outer-text">
   <b>
    Second outer paragraph.
   </b>
  </p>
 </body>
</html>


## Trouver des éléments grâce à leur tag

La grande force de Beautiful Soup est la recherche de contenu. Nous pouvons extraire un élément grâce à son **tag** avec la méthode **`.find('tag')`** :

In [6]:
soup.find('p')

<p class="inner-text first-item" id="first">
                First paragraph.
            </p>

Attention : `find` ne récupère que le premier tag qui remplit la condition. Pour obtenir tous les tags de la page remplissant la condition, il faut utiliser **`find_all`**. Le résultat est sous forme de **liste** : 

In [5]:
soup.find_all('p')

[<p class="inner-text first-item" id="first">
                 First paragraph.
             </p>,
 <p class="inner-text">
                 Second paragraph.
             </p>,
 <p class="outer-text first-item" id="second">
 <b>
                 First outer paragraph.
             </b>
 </p>,
 <p class="outer-text">
 <b>
                 Second outer paragraph.
             </b>
 </p>]

Une fois que le tag souhaité est isolé, `bs` nous permet d'extraire son contenu avec `.get_text()` :

In [6]:
soup.find('p').get_text()

'\n                First paragraph.\n            '

In [9]:
# Bonus : .strip() pour enlever les blancs avant et après le texte extrait
soup.find('p').get_text().strip()

'First paragraph.'

In [7]:
# Attention : .get_text ne fonctionne pas sur une liste ('find_all')
soup.find_all('p').get_text()

AttributeError: ResultSet object has no attribute 'get_text'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [11]:
soup.find_all('p')

[<p class="inner-text first-item" id="first">
                 First paragraph.
             </p>,
 <p class="inner-text">
                 Second paragraph.
             </p>,
 <p class="outer-text first-item" id="second">
 <b>
                 First outer paragraph.
             </b>
 </p>,
 <p class="outer-text">
 <b>
                 Second outer paragraph.
             </b>
 </p>]

In [6]:
# Nous pouvons utiliser une liste en compréhension
texte = [ p.get_text().strip() for p in soup.find_all('p') ]
texte

['First paragraph.',
 'Second paragraph.',
 'First outer paragraph.',
 'Second outer paragraph.']

## Plus précis : trouver des éléments grâce à leurs propriétés

Comment faire pour extraire seulement le second paragraphe ? 

Nous allons utiliser ici les **"attributs"** de chaque tag. 

Ici, tous nos paragraphes `<p>` ont une **`class`** et un **`id`**. Ces attributs sont utilisés pour la mise en page du site, mais nous allons les utiliser pour notre recherche !

In [15]:
soup.find_all('p', {"class":"outer-text"})

[<p class="outer-text first-item" id="second">
 <b>
                 First outer paragraph.
             </b>
 </p>,
 <p class="outer-text">
 <b>
                 Second outer paragraph.
             </b>
 </p>]

In [17]:
soup.find_all('p', {"class":"inner-text"})

[<p class="inner-text first-item" id="first">
                 First paragraph.
             </p>,
 <p class="inner-text">
                 Second paragraph.
             </p>]

In [14]:
soup.find_all('p', {"class":"outer-text", "id":"second"})

[<p class="outer-text first-item" id="second">
 <b>
                 First outer paragraph.
             </b>
 </p>]

In [9]:
soup.find_all('p', {"class":"inner-text", "id":"first"})

[<p class="inner-text first-item" id="first">
                 First paragraph.
             </p>]

In [10]:
# BONUS : un id étant unique en HTML, on peut y accéder directement sans mentionner le tag
soup.find(id="first")

<p class="inner-text first-item" id="first">
                First paragraph.
            </p>

$\rightarrow$ Avec la méthode **`find_all`** et un **dictionnaire d'attributs** (`class` et `id` sont les plus communs, mais il en existe beaucoup d'autres selon le site scrapé), vous allez pouvoir précisément **trouver** le tag souhaité et **extraire** son contenu.